In [1]:
from qiskit import QuantumCircuit
from qiskit.visualization import visualize_transition
from qiskit.visualization import plot_bloch_multivector
import numpy as np

# Classe Sampler e Estimator

Calcular o valor esperado das observ ́aveis X, Y e Z de
um circuito de 1 qubit usando o Sampler e validar os resultados utilizando o
Estimator do Qiskit.

### Passo 1 - Criar o Circuito Quântico:

### Passo 2 - Calcular o Valor Esperado de Z usando o Sampler:

### Passo 3 - Calcular o Valor Esperado de X usando o Sampler:

### Passo 4 - Calcular o Valor Esperado de Y usando o Sampler:

### Passo 5 - Validar os Resultados usando o Estimator:

## FIM